In [1]:
import scrapy, re, urllib, datetime, io, requests
import pandas as pd
from scrapy.crawler import CrawlerProcess
from scrapy.linkextractors import LinkExtractor
from bs4 import BeautifulSoup as bs

In [11]:
class HistorySpider(scrapy.Spider): 
    name = "History"
    
    file_in = "NewLinks2017-12-03 20:28:18.862583.csv"
    file_out = "History_{0}.csv"
    variables = ["Team","Year","Head Coaches","Division","Conference","Wins","Losses","Ties","WL%","Notes"]
    header = ','.join(variables) + "\n"
    target = 'history'
    
    # Read in data
    data = pd.read_csv(file_in)
    df = data[data.key == target].drop_duplicates()
    df.sort_values(['team'], inplace = True)
    _list = df.to_dict(orient='record')
    
    # Create file to append to 
    fileName = file_out.format(datetime.datetime.now())
    
    with open(fileName, 'w') as f: 
        f.write(header)
    
    def start_requests(self): 
        
        for url in self._list[:1]: 
            yield scrapy.Request(url=url['link'], 
                                 callback=self.parse, 
                                 meta={'team':url['team']}
                                )
    
    def parse(self, response):         
        # Create tables 
        tables = pd.read_html(response.body)
        tables[0]['Team'] = response.meta['team']

        # Create a pandas df and append to csv 
        c = io.StringIO()
        tables[0][self.variables][:-1].to_csv(c, index=False, header=False)
        
        # Append links to CSV 
        with open(self.fileName, 'a') as f:
            f.write(c.getvalue())
        

In [12]:
process = CrawlerProcess({
    'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)'
})

process.crawl(HistorySpider)
process.start() 

2017-12-05 22:00:52 [scrapy.utils.log] INFO: Scrapy 1.4.0 started (bot: scrapybot)
2017-12-05 22:00:52 [scrapy.utils.log] INFO: Overridden settings: {'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)'}
2017-12-05 22:00:52 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.memusage.MemoryUsage',
 'scrapy.extensions.logstats.LogStats']
2017-12-05 22:00:52 [scrapy.middleware] INFO: Enabled downloader middlewares:
['scrapy.downloadermiddlewares.httpauth.HttpAuthMiddleware',
 'scrapy.downloadermiddlewares.downloadtimeout.DownloadTimeoutMiddleware',
 'scrapy.downloadermiddlewares.defaultheaders.DefaultHeadersMiddleware',
 'scrapy.downloadermiddlewares.useragent.UserAgentMiddleware',
 'scrapy.downloadermiddlewares.retry.RetryMiddleware',
 'scrapy.downloadermiddlewares.redirect.MetaRefreshMiddleware',
 'scrapy.downloadermiddlewares.httpcompression.HttpCompressionMiddleware',
 'sc